In [1]:
%load_ext autoreload
%autoreload 2

Important constants

In [1]:
NUM_TERMINS = 50

Some frequently used imports

In [2]:
import numpy as np
import pandas as pd

In [3]:
from tqdm.notebook import trange
from tqdm.notebook import tqdm

Load the dataset and lemmatizer

In [4]:
from utils.data import Lemmatizer
from utils.loaders import IMDBDataset

In [5]:
imdb_dataset = IMDBDataset.load()

In [6]:
lemmatizer = Lemmatizer("en")

Fit TfidfVectorizer on the dataset

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
vectorizer = TfidfVectorizer()

In [9]:
texts = []
for text in tqdm(imdb_dataset["train"]["text"]):
    texts.append(lemmatizer.lemmatize_text(text))

  0%|          | 0/25000 [00:00<?, ?it/s]

In [10]:
X = vectorizer.fit_transform(texts)

Count nonzero elements

In [11]:
ans = []
for i in trange(X.shape[1]):
    ans.append(X.getcol(i).count_nonzero())

  0%|          | 0/61782 [00:00<?, ?it/s]

In [12]:
word_nonzero = np.array(ans)

Create mask for stop words 

In [13]:
import nltk
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\znaum\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
stop_mask = np.array([word in stopwords.words("english") for word in vectorizer.get_feature_names_out()])

Get the most frequent termins

In [16]:
df = pd.DataFrame({
    "word": vectorizer.get_feature_names_out(),
    "count": word_nonzero * np.invert(stop_mask)
})

In [17]:
df = df.sort_values("count", ascending=False)

In [18]:
termins = df["word"][:NUM_TERMINS].to_list()

Save the termins

In [19]:
import json

In [20]:
json.dump(termins, open("./termins.json", "w"))